<a href="https://colab.research.google.com/github/B4ben-69/tensorFlow_edging/blob/main/Process_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emd

In [9]:
import emd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.preprocessing import OneHotEncoder

# Load data from JSON, ignoring lines with "wifiStatus"
with open('/content/drive/MyDrive/Colab Notebooks/stream_output_20240902_085854.txt', 'r') as f:
  data = []
  for line in f:
    try:
      item = json.loads(line)
      if 'pressure' in item:  # Assuming your relevant data has "pressure" key
        data.append(item)
    except json.JSONDecodeError:
      pass  # Ignore lines that are not valid JSON

# Now you have a list of dictionaries with relevant data
# Extract pressure values
pressure_values = [item['pressure'] for item in data]
# Extract pressure values
pressure_values = [item['pressure'] for item in data]

window_size = 100  # Adjust as needed
moving_avg = np.convolve(pressure_values, np.ones(window_size), 'same') / window_size
pressure_values_normalized = pressure_values - moving_avg

# Create an EMD object and extract IMFs
imf = emd.sift.sift(np.array(pressure_values_normalized))
imfs = imf[:, 1:]

# Extract other features from JSON data
# Normalize motor values to the range [0, 1]
motor_values = np.array([item['motor'] for item in data]) / 255.0

runMode_values = np.array([item['runMode'] for item in data])
# One-hot encode runMode
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for numpy array
runMode_encoded = encoder.fit_transform(runMode_values.reshape(-1, 1))

# Convert string booleans to integers
permitOrgasm_values = np.array([1 if item['permitOrgasm'] == 'true' else 0 for item in data])

# Create a NumPy array with IMFs and other features
# Ensure all feature arrays have the same length
min_length = min(len(imfs), len(motor_values), len(runMode_values), len(permitOrgasm_values))
#dataset = np.hstack((imfs[:min_length],
#                      motor_values[:min_length].reshape(-1, 1),
#                      runMode_encoded[:min_length],
#                      permitOrgasm_values[:min_length].reshape(-1, 1)))
dataset = np.hstack((imfs,
                      motor_values.reshape(-1, 1),
                      ))


# Print the shape of the dataset
print(dataset.shape, imfs.shape, motor_values.reshape(-1, 1).shape)

(46871, 10) (46871, 9) (46871, 1)


In [10]:
dataset[:100]

array([[ 4.65887178e+01,  3.13628602e+02,  3.36519533e+02,
         1.54857278e+02,  1.97744416e+02,  1.39115287e+02,
         6.35601773e+00,  3.14458629e-01, -1.76284035e+01,
         1.96078431e-02],
       [ 2.88473968e+01,  3.13365205e+02,  3.51569535e+02,
         1.57273221e+02,  1.97809559e+02,  1.39092638e+02,
         6.33530687e+00,  3.11284747e-01, -1.76249306e+01,
         1.96078431e-02],
       [ 1.17509135e+01,  3.14002536e+02,  3.65688804e+02,
         1.59494701e+02,  1.97840787e+02,  1.39067968e+02,
         6.31459485e+00,  3.08114552e-01, -1.76214540e+01,
         1.96078431e-02],
       [-4.40806900e+00,  3.15324024e+02,  3.78818627e+02,
         1.61516569e+02,  1.97837899e+02,  1.39041279e+02,
         6.29388175e+00,  3.04948048e-01, -1.76179737e+01,
         1.96078431e-02],
       [-1.93958639e+01,  3.17172071e+02,  3.90900287e+02,
         1.63333676e+02,  1.97800692e+02,  1.39012579e+02,
         6.27316763e+00,  3.01785238e-01, -1.76144897e+01,
         1.